Large Waste Items:
- Number of detections: 65
- Average confidence: 0.427
- Maximum confidence: 0.912
- Minimum confidence: 0.252

PP bag:
- Number of detections: 44
- Average confidence: 0.557
- Maximum confidence: 0.943
- Minimum confidence: 0.270

General Waste:
- Number of detections: 163
- Average confidence: 0.426
- Maximum confidence: 0.741
- Minimum confidence: 0.250

CleanNet:
- Number of detections: 119
- Average confidence: 0.745
- Maximum confidence: 0.959
- Minimum confidence: 0.265

이걸 기준으로 평균보다 아래는 Red바운딩 박스 이상은 Green 바운딩 박스를 표출한다.

In [9]:
# Cell 1: 필요한 라이브러리 임포트
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import os
import json
from pathlib import Path

# Cell 2: 카테고리 매핑 정의
category_mapping = {
    # 대형폐기물 (Large Waste Items)
    'arcade machine': 'Large Waste Items',
    'Audio': 'Large Waste Items',
    'Computer': 'Large Waste Items',
    'fax machine': 'Large Waste Items',
    'Main unit': 'Large Waste Items',
    'Monitor': 'Large Waste Items',
    'Printer': 'Large Waste Items',
    'sewing machine': 'Large Waste Items',
    'Speaker': 'Large Waste Items',
    'typewriter': 'Large Waste Items',
    'Vacuum cleaner': 'Large Waste Items',
    'Video player': 'Large Waste Items',
    'Bathtub': 'Large Waste Items',
    'Sink': 'Large Waste Items',
    'Kitchen sink': 'Large Waste Items',
    'Toilet bowl': 'Large Waste Items',
    'Bed': 'Large Waste Items',
    'Bookcase': 'Large Waste Items',
    'Bookstand': 'Large Waste Items',
    'Cabinet': 'Large Waste Items',
    'chair': 'Large Waste Items',
    'Cupboard': 'Large Waste Items',
    'Desk': 'Large Waste Items',
    'Dining table': 'Large Waste Items',
    'Display cabinet': 'Large Waste Items',
    'Display stand': 'Large Waste Items',
    'Drawer unit': 'Large Waste Items',
    'Shoe rack': 'Large Waste Items',
    'Small cabinet': 'Large Waste Items',
    'Sofa': 'Large Waste Items',
    'Table': 'Large Waste Items',
    'TV stand': 'Large Waste Items',
    'Vanity table': 'Large Waste Items',
    'Wardrobe': 'Large Waste Items',
    'Air conditioner': 'Large Waste Items',
    'Air purifier': 'Large Waste Items',
    'dish dryer': 'Large Waste Items',
    'Electric rice cooker': 'Large Waste Items',
    'Fan': 'Large Waste Items',
    'Gas oven range': 'Large Waste Items',
    'Heater': 'Large Waste Items',
    'Humidifier': 'Large Waste Items',
    'Microwave': 'Large Waste Items',
    'refrigerator': 'Large Waste Items',
    'Spin dryer': 'Large Waste Items',
    'TV': 'Large Waste Items',
    'Washing machine': 'Large Waste Items',
    'Aquarium': 'Large Waste Items',
    'Bamboo mat': 'Large Waste Items',
    'Bedding items': 'Large Waste Items',
    'bicycle': 'Large Waste Items',
    'Carpet': 'Large Waste Items',
    'Clothes drying rack': 'Large Waste Items',
    'Coat rack': 'Large Waste Items',
    'Door panel': 'Large Waste Items',
    'Earthenware jar': 'Large Waste Items',
    'Floor covering': 'Large Waste Items',
    'Frame': 'Large Waste Items',
    'lumber': 'Large Waste Items',
    'Mannequin': 'Large Waste Items',
    'Mat': 'Large Waste Items',
    'Piano': 'Large Waste Items',
    'Rice storage container': 'Large Waste Items',
    'Signboard': 'Large Waste Items',
    'Stroller': 'Large Waste Items',
    'Wall clock': 'Large Waste Items',
    'Water tank': 'Large Waste Items',
    'audio cabinet': 'Large Waste Items',
    'suitcase': 'Large Waste Items',
    
    # 기타 카테고리
    'PP bag': 'PP bag',
    'General waste bag': 'General Waste',
    'waste pile': 'General Waste',
    'CleanNet': 'CleanNet',
    # 원본 클래스명도 추가
    'General Waste': 'General Waste'  # 원본 클래스명 매핑 추가
}

# Cell 3: 바운딩 박스 색상 결정 함수
def get_bbox_color(class_name, confidence):
    # 먼저 클래스명이 매핑에 있는지 확인하고 매핑된 카테고리 사용
    mapped_class = category_mapping.get(class_name, class_name)
    
    if mapped_class == 'Large Waste Items':
        return (0, 255, 0) if confidence > 0.4 else (0, 0, 255)  # 초록색 또는 빨간색
    elif mapped_class == 'PP bag':
        return (0, 255, 0) if confidence > 0.5 else (0, 0, 255)  # 초록색 또는 빨간색
    else:
        return (255, 255, 255)  # 흰색 (General Waste와 CleanNet)

# Cell 4: 객체 감지 및 시각화 함수
def detect_objects(model_path, image_dir):
    # 모델 로드
    model = YOLO(model_path)
    
    # 출력 디렉토리 생성
    output_dir = './runs/detect/predict/'
    os.makedirs(output_dir, exist_ok=True)
    
    # 모든 이미지 파일 가져오기
    image_paths = []
    for ext in ['*.jpg', '*.jpeg', '*.png']:
        image_paths.extend(list(Path(image_dir).glob(ext)))
    
    print(f"Found {len(image_paths)} images in the directory.")
    
    results = []
    for idx, image_path in enumerate(image_paths, 1):
        print(f"\nProcessing image {idx}/{len(image_paths)}: {image_path.name}")
        
        # 이미지 로드 및 객체 감지
        image = cv2.imread(str(image_path))
        if image is None:
            print(f"Error: Could not load image {image_path}")
            continue
            
        result = model.predict(source=str(image_path), save=False, device='cpu')[0]
        
        # 결과 시각화
        for box, conf, cls_id in zip(result.boxes.xyxy, result.boxes.conf, result.boxes.cls):
            x1, y1, x2, y2 = box.cpu().numpy().astype(int)
            confidence = conf.cpu().numpy()
            class_name = result.names[int(cls_id)]
            
            try:
                # 바운딩 박스 색상 결정
                color = get_bbox_color(class_name, confidence)
                
                # 바운딩 박스 그리기
                cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
                
                # 클래스 이름과 신뢰도 표시
                label = f'{class_name} {confidence:.2f}'
                cv2.putText(image, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            except Exception as e:
                print(f"Error processing detection for class {class_name}: {e}")
                continue
        
        # 결과 이미지 저장
        output_path = os.path.join(output_dir, f'{Path(image_path).stem}_detected{Path(image_path).suffix}')
        cv2.imwrite(output_path, image)
        
        results.append(result)
        
        # 감지된 클래스 출력
        detected_classes = [result.names[int(cls_id)] for cls_id in result.boxes.cls]
        print(f"Detected Classes in {image_path.name}:")
        for cls in detected_classes:
            print(f"- {cls}")
        
        # 처리 속도 정보 출력
        preprocess_time = result.speed['preprocess']
        inference_time = result.speed['inference']
        postprocess_time = result.speed['postprocess']
        print(f"Processing times:")
        print(f"- Preprocess: {preprocess_time}ms")
        print(f"- Inference: {inference_time}ms")
        print(f"- Postprocess: {postprocess_time}ms")
    
    print(f"\nAll results saved to {output_dir}")
    return results

# Cell 5: JSON 라벨 변환 함수
def convert_labels_to_major_category(json_path, category_mapping):
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    converted_data = {}
    for image_name, annotations in data.items():
        converted_annotations = []
        for ann in annotations:
            class_name = ann["class_name"]
            if class_name in category_mapping:
                major_category = category_mapping[class_name]
                converted_annotations.append({
                    "class_name": major_category,
                    "bbox": ann["bbox"]
                })
        converted_data[image_name] = converted_annotations
    
    return converted_data

# Cell 6: 이진 분류 함수
def classify_images_by_major_category(converted_labels, target_class):
    binary_classification = {}
    
    for image_name, annotations in converted_labels.items():
        if any(ann["class_name"] == target_class for ann in annotations):
            binary_classification[image_name] = "Positive"
        else:
            binary_classification[image_name] = "Negative"
            
        # 상세 정보 출력
        detected_classes = [ann["class_name"] for ann in annotations]
        print(f"\nImage: {image_name}")
        print(f"Detected classes: {detected_classes}")
        print(f"Classification: {binary_classification[image_name]}")
    
    return binary_classification

# Cell 7: 결과 저장 함수
def save_results(binary_classification, output_path):
    results_summary = {
        "total_images": len(binary_classification),
        "positive_count": sum(1 for v in binary_classification.values() if v == "Positive"),
        "negative_count": sum(1 for v in binary_classification.values() if v == "Negative"),
        "classifications": binary_classification
    }
    
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(results_summary, f, indent=4, ensure_ascii=False)
    
    return results_summary

# Cell 8: 메인 실행 코드
def main():
    # 경로 설정
    model_path = r'E:\livingLab\YOLOv8기반 객체인식\results_20241124_155336\waste_detection_model\weights\best.pt'
    image_dir = r'E:\livingLab\YOLOv8기반 객체인식\test\images'
    json_path = r'E:\livingLab\YOLOv8기반 객체인식\test\test_annotations_yolo.json'
    target_class = 'Large Waste Items'
    
    # 1. 객체 감지 실행
    print("Starting object detection...")
    results = detect_objects(model_path, image_dir)
    
    # 2. 라벨 변환
    print("\nConverting labels to major categories...")
    converted_labels = convert_labels_to_major_category(json_path, category_mapping)
    
    # 3. 이진 분류 실행
    print("\nPerforming binary classification...")
    binary_classification = classify_images_by_major_category(converted_labels, target_class)
    
    # 4. 결과 저장
    output_path = './test/classification_results.json'
    results_summary = save_results(binary_classification, output_path)
    
    # 5. 최종 결과 출력
    print("\nClassification Summary:")
    print(f"Total images processed: {results_summary['total_images']}")
    print(f"Positive classifications: {results_summary['positive_count']}")
    print(f"Negative classifications: {results_summary['negative_count']}")
    print(f"\nDetailed results saved to: {output_path}")

if __name__ == "__main__":
    main()

Starting object detection...
Found 79 images in the directory.

Processing image 1/79: -7-_jpg.rf.3e4e45f01df8da1ac999a94efb60473e.jpg

image 1/1 E:\livingLab\YOLOv8 \test\images\-7-_jpg.rf.3e4e45f01df8da1ac999a94efb60473e.jpg: 640x640 1 PP bag, 41.0ms
Speed: 1.8ms preprocess, 41.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
Detected Classes in -7-_jpg.rf.3e4e45f01df8da1ac999a94efb60473e.jpg:
- PP bag
Processing times:
- Preprocess: 1.8334388732910156ms
- Inference: 40.97270965576172ms
- Postprocess: 0.0ms

Processing image 2/79: 202411061313-36-307488-127-352668_jpg.rf.2debbe814df92503a0079be859650cc0.jpg

image 1/1 E:\livingLab\YOLOv8 \test\images\202411061313-36-307488-127-352668_jpg.rf.2debbe814df92503a0079be859650cc0.jpg: 640x640 1 General Waste, 2 CleanNets, 35.8ms
Speed: 4.0ms preprocess, 35.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Detected Classes in 202411061313-36-307488-127-352668_jpg.rf.2debbe814df92503a0079be859650cc0.jpg: